In [1]:
%%configure -f
{
  "conf": {
      "spark.sql.extensions":  "io.delta.sql.DeltaSparkSessionExtension",
      "spark.sql.catalog.spark_catalog":  "org.apache.spark.sql.delta.catalog.DeltaCatalog",
      "spark.jars": "file:///usr/share/aws/delta/lib/delta-storage.jar,file:///usr/share/aws/delta/lib/delta-spark.jar",
      "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog"
  }
}

In [ ]:
spark

In [3]:
sts = boto3.client('sts')
response = sts.get_caller_identity()
print(response)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'UserId': 'AROATKTHTIZOAAMGOZCQO:00fhtkhd7vv1kl09,00fhtnkn92tqt10a', 'Account': '228924278364', 'Arn': 'arn:aws:sts::228924278364:assumed-role/emrserverless-jobrole-sony-demo/00fhtkhd7vv1kl09,00fhtnkn92tqt10a', 'ResponseMetadata': {'RequestId': '24929d24-1534-40c2-8d36-c993fd6e96f1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '24929d24-1534-40c2-8d36-c993fd6e96f1', 'content-type': 'text/xml', 'content-length': '504', 'date': 'Tue, 19 Mar 2024 18:21:44 GMT'}, 'RetryAttempts': 0}}

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import HiveContext, SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from delta import *
from numpy import exp
import boto3
import builtins
import json
import math
import os
import pandas as pd
import numpy as np
import logging
import time

VBox()

Starting Spark application


ID,Kind,State,Spark UI,Driver log,User,Current session?
3,pyspark,idle,Link,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
BUCKET = "amzn-customer-reviews-228924278364"
#PREFIX = "amzn-customer-reviews-partitioned/category=Books/review_year=2014/review_month=10/review_day=10/"
PREFIX = "delta/amznreviews/books/"
PATH = f"s3a://{BUCKET}/{PREFIX}"
REGION = "us-east-1"

print(PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3a://amzn-customer-reviews-228924278364/delta/amznreviews/books/

In [5]:
def getBodyJson(system_prompt, message_json):
    body_summary = json.dumps({
        "max_tokens": 2048,
        "temperature": 0.5,
        "system": system_prompt,
        "messages": [ message_json ],
        "anthropic_version": "bedrock-2023-05-31"
    })

    return body_summary

def getMessageJson(user_prompt):
    user_prompt = f"""\n\n\nHuman: {user_prompt}\n\nAssistant:"""
    message_json = {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": user_prompt
            }
        ]
    }
    return message_json

def getPrompt(type,txt):
    if type == "summary":
        user_prompt = f"""Write a summary of the following review; all your output has to be grammatically correct in a single paragraph:\n{txt}"""
    elif type == "sentiment":
        user_prompt = f"""The text below is a book review. Tell me the sentiment of the text here:\n{txt}\n The sentiment categories are: \n(1) Positive\n(2) Negative\n(3) Neutral"""
    elif type == "sentiment_one_label":
        user_prompt = f"""The text below is a book review.\nTell me the sentiment of the text here by returning only one label, which can be (1) Positive, (2) Negative, or (3) Neutral.\nThe text is here:\n{txt}\n"""
    else:
        user_prompt = "error"

    return user_prompt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def getPdfAnthropic(df:pd.DataFrame)-> pd.DataFrame:

    # Define model ID
    model_id = 'anthropic.claude-3-haiku-20240307-v1:0'
    # Set system prompt
    system_prompt = "The texts here are all book reviews."

    # Instantiate bedrock
    client_bedrock = boto3.client("bedrock-runtime", REGION)

    def getAnthropic(type,txt):
        user_prompt = getPrompt(type,txt)

        if user_prompt == "error":
            return "error"
        else:
            message_json = getMessageJson(user_prompt)
            body = getBodyJson(system_prompt, message_json)
        
            response = client_bedrock.invoke_model(modelId=model_id, body=body)
            response_body_summary = json.loads(response["body"].read())
            response_summary = response_body_summary["content"][0]["text"]

        return response_summary

    
    reviewText = [d.reviewText for idx, d in df.iterrows()]

    anthropicSummary_list = []
    
    for r in reviewText:
        each_summary = getAnthropic("summary",r)
        anthropicSummary_list.append(each_summary)

    anthropicSummary_array = np.array([anthropicSummary_list])
    anthropicSummary_concat = np.concatenate(anthropicSummary_array)

    return_df = (
        df[[
            "asin","overall","reviewText",
            "reviewTimeTS", "reviewerID",
            "reviewerName", "summary",
            "verified", "row_id",
            "asin_key", "row_number"
          ]]
        .assign(anthropicSummary=list(anthropicSummary_concat))
    )

    return return_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df = spark.read.format("delta").load(PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTimeTS: timestamp (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- row_number: integer (nullable = true)
 |-- asin_key: integer (nullable = true)

In [9]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

23184

In [10]:
 # Create row_count
df = df.withColumn("temp_column", lit("A"))
w = Window().partitionBy("temp_column").orderBy(lit("A"))
df = df.withColumn("row_id", row_number().over(w)).drop("temp_column")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTimeTS: timestamp (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- row_number: integer (nullable = true)
 |-- asin_key: integer (nullable = true)
 |-- row_id: integer (nullable = false)

In [12]:
df_test = df.limit(30)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
schema = StructType(
       [
           StructField('asin', StringType(), True),
           StructField('overall', DoubleType(), True),
           StructField('reviewText', StringType(), True),
           StructField('reviewTimeTS', TimestampType(), True),
           StructField('reviewerID', StringType(), True),
           StructField('reviewerName', StringType(), True),
           StructField('summary', StringType(), True),
           StructField('verified', BooleanType(), True),
           StructField('row_id', IntegerType(), True),
           StructField('row_number', IntegerType(), True),
           StructField('asin_key', IntegerType(), True),
           StructField("anthropicSummary", StringType(), True)
       ]
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_summary = ( df_test
      .groupBy(spark_partition_id().alias("_pid"))
      .applyInPandas(getPdfAnthropic,schema)
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# Get start time
t0 = time.localtime()
start_time = time.strftime("%H:%M:%S", t0)
print(start_time)

# Get Dataframe
df_summary.cache()
df_summary.show()

# Get end time
t1 = time.localtime()
end_time = time.strftime("%H:%M:%S", t1)
print(end_time)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

18:24:38
+----------+-------+--------------------+-------------------+--------------+-------------------+--------------------+--------+------+----------+--------+--------------------+
|      asin|overall|          reviewText|       reviewTimeTS|    reviewerID|       reviewerName|             summary|verified|row_id|row_number|asin_key|    anthropicSummary|
+----------+-------+--------------------+-------------------+--------------+-------------------+--------------------+--------+------+----------+--------+--------------------+
|0002005107|    5.0|Thumps DreadfulWa...|2008-07-13 00:00:00| AACMLWNILA6CL|Diana Tixier Herald|         Love Thumps|   false|     1|         1| 2005107|The review descri...|
|0006280544|    3.0|Mere Chistianity ...|2008-07-13 00:00:00|A1W98Q4MZK80VA|Joseph P. Tevington|Nothing "Mere" Ab...|   false|     2|         1| 6280544|The review summar...|
|0007127898|    5.0|Mabinogion myth m...|2008-07-13 00:00:00|A1VCUE373CMQBY|    Amazon Customer|Let this story bl...

In [16]:
# Difference 
diff_time = ( time.mktime(t1) - time.mktime(t0) )
print(f"Difference (seconds): {diff_time}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Difference (seconds): 76.0

In [17]:
df_summary_text = df_summary.select("reviewText", "anthropicSummary").collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
print(df_summary_text[1]["anthropicSummary"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The review summarizes the key ideas presented in C.S. Lewis' book "Mere Christianity," which is divided into four books. In Book 1, Lewis argues against relativistic thinking, asserting that there are universal moral standards that come from God. In Book 2, he addresses the divinity of Jesus, stating that if Jesus was merely a man, he would either be a lunatic or the devil. Book 3 discusses the "cardinal virtues" and the importance of forgiveness and humility. The review notes that Lewis' commentary on sexual morality was prophetic, though he also made some questionable statements about marriage. In Book 4, Lewis warns against a "vague religion" and emphasizes the importance of submitting to Christ to truly find one's own personality. Overall, the review highlights the depth and significance of the ideas presented in this influential work by C.S. Lewis.